#Use Exit to end conversations

In [1]:
# Install TextBlob for sentiment analysis
!pip install textblob
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [ ]:
import sys
import random
from textblob import TextBlob

class SentimentChatbot:
    def __init__(self):
        self.conversation_history = []

    def analyze_sentiment(self, text):
        """
        Tier 2 Requirement: Statement-Level Analysis
        Returns: (Label, Polarity)
        """
        analysis = TextBlob(text)
        polarity = analysis.sentiment.polarity

        if polarity > 0.1:
            return "Positive", polarity
        elif polarity < -0.1:
            return "Negative", polarity
        else:
            return "Neutral", polarity

    def generate_response(self, text, sentiment_label):
        """
         IMPROVED LOGIC:
         Selects a response based on the detected sentiment and randomizes
         the output to avoid repetition.
        """
        text_lower = text.lower()

        # 1. Check for specific keywords first (Rule-based)
        if "hello" in text_lower or "hi" in text_lower:
            return "Hello there! How are you feeling today?"
        if "help" in text_lower:
            return "I am here to listen. Tell me what's on your mind."

        # 2. If no keywords, respond based on Sentiment (Context-aware)
        if sentiment_label == "Positive":
            responses = [
                "That sounds wonderful!",
                "I'm glad to hear that! Tell me more.",
                "It's great to see you in such a good mood.",
                "That's excellent news!",
                "Keep those positive vibes coming!"
            ]
        elif sentiment_label == "Negative":
            responses = [
                "I'm sorry to hear that. That sounds tough.",
                "I understand your frustration. I'm listening.",
                "That must be disappointing.",
                "I appreciate you sharing that with me, even if it's hard.",
                "Let's hope things get better soon."
            ]
        else: # Neutral
            responses = [
                "I see. Please go on.",
                "Could you elaborate on that?",
                "Interesting. Tell me more.",
                "I'm listening.",
                "Okay, noted."
            ]

        return random.choice(responses)

    def show_summary(self):
        """
        Tier 1: Conversation Summary
        """
        if not self.conversation_history:
            print("\nNo conversation to analyze.")
            return

        print("\n" + "="*40)
        print("FINAL CONVERSATION REPORT")
        print("="*40)

        avg_polarity = sum(p for _, _, p in self.conversation_history) / len(self.conversation_history)

        if avg_polarity > 0.1:
            mood = "Positive - Generally Satisfied"
        elif avg_polarity < -0.1:
            mood = "Negative - Generally Dissatisfied"
        else:
            mood = "Neutral - Objective/Balanced"

        print(f"Overall Conversation Sentiment: {mood}")
        print(f"Average Polarity Score: {avg_polarity:.2f}")

    def run(self):
        print("Bot: Hello! I am ready to chat. (Type 'exit' to quit)\n")

        while True:
            try:
                user_input = input("User: ")
                if user_input.lower() in ['exit', 'quit', 'bye']:
                    break

                # Analyze
                sentiment_label, polarity = self.analyze_sentiment(user_input)

                # Tier 2 Output
                print(f"→ Sentiment: {sentiment_label} ({polarity:.2f})")

                # Store
                self.conversation_history.append((user_input, sentiment_label, polarity))

                # Generate better response using the sentiment label
                response = self.generate_response(user_input, sentiment_label)
                print(f"Chatbot: {response}\n")

            except KeyboardInterrupt:
                break

        self.show_summary()

if __name__ == "__main__":
    bot = SentimentChatbot()
    bot.run()

Bot: Hello! I am ready to chat. (Type 'exit' to quit)

User: hello 
→ Sentiment: Neutral (0.00)
Chatbot: Hello there! How are you feeling today?

User: bad 
→ Sentiment: Negative (-0.70)
Chatbot: Let's hope things get better soon.

User: how >
→ Sentiment: Neutral (0.00)
Chatbot: Interesting. Tell me more.

User: ? it have been a teriible experience
→ Sentiment: Neutral (0.00)
Chatbot: Okay, noted.

User: teribal
→ Sentiment: Neutral (0.00)
Chatbot: I'm listening.

User: i wanna say you suck
→ Sentiment: Neutral (0.00)
Chatbot: Interesting. Tell me more.

User: why 
→ Sentiment: Neutral (0.00)
Chatbot: I see. Please go on.

User: where ?
→ Sentiment: Neutral (0.00)
Chatbot: Okay, noted.

User: fuck u
→ Sentiment: Negative (-0.40)
Chatbot: Let's hope things get better soon.

User: can you help me 
→ Sentiment: Neutral (0.00)
Chatbot: I am here to listen. Tell me what's on your mind.

User: i wana be a engineer
→ Sentiment: Neutral (0.00)
Chatbot: Interesting. Tell me more.

User: how ca

# USING Gemini Api (Write Exit to generate summary)


In [2]:
!pip install -q -U google-generativeai

In [8]:
import google.generativeai as genai
from textblob import TextBlob
import getpass
import os
import sys

class GeminiSentimentBot:
    def __init__(self):
        # 1. Securely get API Key
        if "GOOGLE_API_KEY" not in os.environ:
            print("Please enter your Google API Key (input will be hidden):")
            os.environ["GOOGLE_API_KEY"] = getpass.getpass()

        genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

        # 2. Initialize Model - Switching to 'gemini-pro' as it is the most stable
        try:
            self.model = genai.GenerativeModel('models/gemini-flash-lite-latest')
            self.chat_session = self.model.start_chat(history=[])
        except Exception as e:
            print(f"Error initializing Gemini: {e}")
            sys.exit(1)

        # 3. System Instruction (Context)
        # We send this as the first message because gemini-pro handles context differently
        initial_context = (
            "You are a helpful customer service assistant. "
            "If the user input is negative, be empathetic. "
            "If the user input is positive, be enthusiastic. "
            "Keep responses concise."
        )
        try:
            self.chat_session.send_message(initial_context)
        except:
            # Sometimes a fresh chat doesn't like system prompts immediately, we can skip
            pass

        self.conversation_history = []

    def analyze_sentiment(self, text):
        """
        Tier 2 Requirement: Statement-Level Analysis
        """
        analysis = TextBlob(text)
        polarity = analysis.sentiment.polarity

        if polarity > 0.1:
            return "Positive", polarity
        elif polarity < -0.1:
            return "Negative", polarity
        else:
            return "Neutral", polarity

    def show_summary(self):
        """
        Tier 1 Requirement: Final Report
        """
        if not self.conversation_history:
            print("\nNo conversation to analyze.")
            return

        print("\n" + "="*40)
        print("FINAL CONVERSATION REPORT")
        print("="*40)

        avg_polarity = sum(p for _, _, p in self.conversation_history) / len(self.conversation_history)

        if avg_polarity > 0.1:
            mood = "Positive - Generally Satisfied"
        elif avg_polarity < -0.1:
            mood = "Negative - Generally Dissatisfied"
        else:
            mood = "Neutral - Objective/Balanced"

        print(f"Overall Conversation Sentiment: {mood}")
        print(f"Average Polarity Score: {avg_polarity:.2f}")

    def run(self):
        print("\n" + "-"*50)
        print("Bot (Gemini Pro): Hello! I'm your AI assistant. How can I help?")
        print("-" * 50 + "\n")

        while True:
            try:
                user_input = input("User: ")
                if user_input.lower() in ['exit', 'quit', 'bye']:
                    print("Bot: Goodbye! Analyzing session data...")
                    break

                # --- Tier 2 Analysis (Local) ---
                sentiment_label, polarity = self.analyze_sentiment(user_input)
                print(f"→ Sentiment: {sentiment_label} (Score: {polarity:.2f})")

                # Store data
                self.conversation_history.append((user_input, sentiment_label, polarity))

                # --- Innovation: AI Response (Cloud) ---
                # Generate response
                response = self.chat_session.send_message(user_input)

                print(f"Chatbot: {response.text}\n")

            except KeyboardInterrupt:
                break
            except Exception as e:
                print(f"An error occurred: {e}")
                # Fallback response if API fails mid-chat
                print("Chatbot: I'm having trouble connecting to my brain, but I'm listening.\n")

        self.show_summary()

if __name__ == "__main__":
    bot = GeminiSentimentBot()
    bot.run()


--------------------------------------------------
Bot (Gemini Pro): Hello! I'm your AI assistant. How can I help?
--------------------------------------------------

User: hi
→ Sentiment: Neutral (Score: 0.00)
Chatbot: Hello! How can I help you today?

User: my product just broke
→ Sentiment: Neutral (Score: 0.00)
Chatbot: Oh no, I'm so sorry to hear that your product broke! That sounds really frustrating. Can you tell me a little more about what happened so I can try to help you find a solution?

User: i am sad
→ Sentiment: Negative (Score: -0.50)
Chatbot: I completely understand why you're feeling sad right now. Dealing with a broken item is never easy. Let's see what we can do to make this better!

User: and feeling bad
→ Sentiment: Negative (Score: -0.70)
Chatbot: I hear you, and I truly empathize with you feeling bad. Please know that we are here to support you through this. Tell me, what can I do *right now* to assist you with the broken product?

User: can you help me ?
→ Sent